In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt

def pca_dimensionality_reduction(indep_X, dep_Y, n):
    # Scaling the data
    scaler = StandardScaler()
    indep_X_scaled = scaler.fit_transform(indep_X)

    # PCA apply panni top 'n' components edukkum
    pca = PCA(n_components=n)
    selected_features = pca.fit_transform(indep_X_scaled)

    # Explained Variance Return panna
    explained_variance = pca.explained_variance_ratio_
    return selected_features, explained_variance

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test, y_train, y_test

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, cm

def logistic(X_train, y_train, X_test, y_test):
    classifier = LogisticRegression(random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def svm_NL(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='rbf', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Navie(X_train, y_train, X_test, y_test):
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def knn(X_train, y_train, X_test, y_test):
    classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def Decision(X_train, y_train, X_test, y_test):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def random(X_train, y_train, X_test, y_test):
    classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

def compare_model_accuracies(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf): 
    # DataFrame-ku 1 row dhan irukku, so list-oda 1st value eduthuko
    dataframe = pd.DataFrame(
        {
            'Logistic': [acclog[0]], 
            'SVMl': [accsvml[0]], 
            'SVMnl': [accsvmnl[0]], 
            'KNN': [accknn[0]], 
            'Navie': [accnav[0]], 
            'Decision': [accdes[0]], 
            'Random': [accrf[0]]
        },
        index=['PCA Results']  # Index name fix panni
    )
    return dataframe

In [2]:
# Dataset preparation
dataset1 = pd.read_csv("churn_preprocess_data.csv", index_col=None)
df2 = dataset1

# Preprocessing
df2 = pd.get_dummies(df2, drop_first=True)
df2.replace({True: 1, False: 0}, inplace=True)

indep_X = df2.drop('Churn_Yes', axis=1)
dep_Y = df2['Churn_Yes']

In [3]:
df2

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card,PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_Yes
0,0,5,27.43,137.15,1,1,1,0,0,1,...,0,1,0,0,0,1,1,0,0,1
1,0,42,38.28,1607.76,0,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1
2,0,61,106.44,6492.84,1,0,0,1,1,0,...,1,0,0,1,0,0,0,0,0,1
3,1,22,92.49,2034.78,1,0,1,1,0,0,...,0,0,1,0,0,0,0,1,0,0
4,1,21,19.63,412.23,1,1,1,0,0,0,...,0,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,61,42.52,2593.72,0,0,1,0,0,1,...,0,0,0,1,0,1,0,1,0,0
296,0,49,22.31,1093.19,0,0,0,1,1,0,...,1,0,1,0,0,1,0,0,1,1
297,0,71,59.57,4229.47,1,0,1,1,0,1,...,0,0,1,0,1,0,0,0,1,1
298,1,1,31.32,31.32,1,1,1,1,0,1,...,1,0,1,0,1,0,1,0,0,0


In [4]:
selected_features, explained_variance = pca_dimensionality_reduction(indep_X, dep_Y, 1)

# Model Training and Evaluation
acclog = []
accsvml = []
accsvmnl = []
accknn = []
accnav = []
accdes = []
accrf = []

In [5]:
X_train, X_test, y_train, y_test = split_scalar(selected_features, dep_Y)

classifier, Accuracy, report, cm = logistic(X_train, y_train, X_test, y_test)
acclog.append(Accuracy)

classifier, Accuracy, report, cm = svm_linear(X_train, y_train, X_test, y_test)
accsvml.append(Accuracy)

classifier, Accuracy, report, cm = svm_NL(X_train, y_train, X_test, y_test)
accsvmnl.append(Accuracy)

classifier, Accuracy, report, cm = knn(X_train, y_train, X_test, y_test)
accknn.append(Accuracy)

classifier, Accuracy, report, cm = Navie(X_train, y_train, X_test, y_test)
accnav.append(Accuracy)

classifier, Accuracy, report, cm = Decision(X_train, y_train, X_test, y_test)
accdes.append(Accuracy)

classifier, Accuracy, report, cm = random(X_train, y_train, X_test, y_test)
accrf.append(Accuracy)

# Classification Results
result = compare_model_accuracies(acclog, accsvml, accsvmnl, accknn, accnav, accdes, accrf)


C:\Users\anandha rishi\anaconda3\anaconda4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anandha rishi\anaconda3\anaconda4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\anandha rishi\anaconda3\anaconda4\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
result
#1

,Logistic,SVMl,SVMnl,KNN,Navie,Decision,Random
PCA Results,0.466667,0.586667,0.426667,0.493333,0.466667,0.573333,0.573333
